In [21]:
# Import ML libraries
import pycaret
import xgboost

# Snowpark for Python
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F

# Import Misc
import json
import pandas as pd


In [22]:
# import psutil
# def cpu_count_patched(logical=False):
#     import os
#     return os.cpu_count()
# psutil.cpu_count = cpu_count_patched

In [24]:
# Create Snowflake Session object
connection_parameters = json.load(open("connection.json"))
session = Session.builder.configs(connection_parameters).create()

OperationalError: 250001: Could not connect to Snowflake backend after 2 attempt(s).Aborting